In [ ]:
!git clone https://gitlab.com/hybit1/massca.git

In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
def square(data, col='steptime', func=np.mean, alg=['cohda', 'swarm', 'default']):
    a = np.arange(len(alg) ** 2, dtype=float).reshape(len(alg), len(alg))
    for x, i in enumerate(alg):
      for y, j in enumerate(alg):
        df = data[(i, j)]
        a[x, y] = func(data[(i, j)][col])
    return pd.DataFrame(a, index=alg, columns=alg)

In [6]:
base_dir = '/content/massca/examples/results'
alg = ['default', 'cohda', 'swarm']
data = {}
for i in alg: # --within {i} --between {j}
  for j in alg:
    df = pd.read_csv(os.path.join(base_dir, f"{i}_{j}_results.csv")).rename(columns={
        'MASSCA-0.MosaikAgent-steptime' : 'steptime',
        'GridSim-0.ExternalGrid-0-value' : 'ExternalGrid',
    })
    df['steptime'] = df['steptime'].shift(1)
    df['production'] = df['GridSim-0.Bus-3-value'] + df['GridSim-0.Bus-5-value']
    df['consumption'] = df['GridSim-0.Bus-2-value'] + df['GridSim-0.Bus-4-value']
    df = df.iloc[1::2]
    data[(i, j)] = df

In [7]:
table = []
for i in alg:
  for j in alg:
    df = data[(i, j)]
    table.append({'within' : i,
                  'between' : j,
                  'runtime' : f"{df['steptime'].mean():.2f}",
                  'grid balance' : f"{df['ExternalGrid'].sum():.2f}",
                  'total production' : f"{df['production'].sum():.2f}",
                  'total consumption' : f"{df['consumption'].sum():.2f}"})
pd.DataFrame(table)

,within,between,runtime,grid balance,total production,total consumption
0,default,default,0.04,0.20,-40.68,40.68
1,default,cohda,4.80,-6.92,-47.84,40.68
2,default,swarm,0.42,0.20,-40.68,40.68
3,cohda,default,2.68,0.23,-40.67,40.68
4,cohda,cohda,5.17,-5.13,-46.04,40.68
5,cohda,swarm,2.83,0.22,-40.68,40.68
6,swarm,default,0.24,6.95,-33.91,40.68
7,swarm,cohda,2.86,5.97,-34.90,40.68
8,swarm,swarm,0.69,6.95,-33.91,40.68


In [8]:
square(data)

,cohda,swarm,default
cohda,5.167412,2.829362,2.683034
swarm,2.859398,0.685871,0.243536
default,4.802485,0.419579,0.036232


In [9]:
square(data, 'ExternalGrid', np.sum)

,cohda,swarm,default
cohda,-5.129387,0.218549,0.226366
swarm,5.967039,6.952969,6.952969
default,-6.924497,0.201138,0.201138


In [10]:
square(data, 'production', np.sum)

,cohda,swarm,default
cohda,-46.042146,-40.681242,-40.673386
swarm,-34.899477,-33.910011,-33.910011
default,-47.835913,-40.682692,-40.682692


In [11]:
square(data, 'consumption', np.sum)

,cohda,swarm,default
cohda,40.682692,40.682692,40.682692
swarm,40.682692,40.682692,40.682692
default,40.682692,40.682692,40.682692
